## FONCTION QUI EXECUTE TOUT CE QUI A ETE FAIT PRECEDEMMENT ET RENVOIE LE STATUT FUMEUR

In [2]:
## ON RECUPERE LES CAS CLINIQUES

import os

# Dossier contenant les fichiers texte de cas cliniques
output_folder = 'Cas_cliniques'
output_folder = 'output2_v1'

# Pn récupère tous les fichiers texte dans le dossier
txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]

# On trie les fichiers par ordre alphabétique (et donc par ordre chronologique de création)
txt_files_sorted = sorted(txt_files)

textes = []
# On ouvre et on extrait le texte des 50 premiers fichiers
for i in range(len(txt_files_sorted)):
    file_path = os.path.join(output_folder, txt_files_sorted[i])
    with open(file_path, 'r') as f:
        text = f.read()
    textes.append(text)

In [3]:
len(textes)

1134

In [4]:
## AFFICHAGE DES CAS CLINIQUES
for t in textes[0:50]:
    #print(t)
    pass

In [5]:
from unidecode import unidecode
import re

def preprocess_text(text):
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)
    text = re.sub(r'/d°', 'deg', text)

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    ascii_text = re.sub(r'\s+', ' ', ascii_text)

    return ascii_text
texte = "A l’examen, on trouvait un tableau d’angiocholite, fait d’une fièvre à 38°8, un ictère cutanéo-muqueux, et une défense de l’hypochondre droit, signe de Murphy positif"

print(texte)
preprocess_text(texte)

A l’examen, on trouvait un tableau d’angiocholite, fait d’une fièvre à 38°8, un ictère cutanéo-muqueux, et une défense de l’hypochondre droit, signe de Murphy positif


"A l'examen, on trouvait un tableau d'angiocholite, fait d'une fievre a 38deg8, un ictere cutaneo-muqueux, et une defense de l'hypochondre droit, signe de Murphy positif"

In [21]:
for cas in textes[0:50]:
    cas = preprocess_text(cas)
    statut_tabagisme,statut_marital,statut_alcool, dico_vide = classify_status(cas)
    if dico_vide == False:
        print(cas)
        print(f"Tabac:{statut_tabagisme}")
        print(f"Alcool:{statut_alcool}")
        print(f"Situation:{statut_marital}")
        print("\n")

Texte : antecedents, Is_negated : True, other_detected : False
Il s'agit d'un garcon de 12 ans, sans antecedents, victime d'une chute de sa hauteur avec impact au niveau de l'hypochondre droit. L'examen trouve un patient apyretique, bon etat hemodynamique, avec une sensibilite abdominale generalisee et un maximum au niveau de l'hypochondre droit. Le bilan biologique objective une hyperleucocytose a 11 200 E/mm3, et une hypereosinophilie a 2839 E/mm3. L'echographie montre un epanchement intraperitoneal de grande abondance, avec un kyste hydatique type II de douze cm, rompu. La TDM abdominale confirme les donnees de l'echographie. Le patient est mis sous triple antibiotherapie, et a beneficie d'une cure chirurgicale. Les suites operatoires sont simples, le patient a ete mis sous cure d'albendazole pendant 6 mois, qu'il n'a suivi que pendant trois mois, avec bonne evolution. Le recul est de 4 ans et demi. 
Tabac:NON-FUMEUR
Alcool:NON-ALCOOLIQUE
Situation:UNKNOWN


Texte : antecedents, Is_

Texte : mariee, Is_negated : False, other_detected : False
Texte : divorce, Is_negated : False, other_detected : False
Texte : divorce, Is_negated : False, other_detected : False
Cas clinique 1 Monsieur A est age de 70 ans, originaire et demeurant a Sfax. Il est issu d'un mariage non consanguin et est le huitieme d'une fratrie de neuf (5 garcons et 4 filles). Il fait partie d'une famille modeste et assez conservatrice. Son pere a ete macon. Sa mere a ete femme au foyer. Il a ete scolarise jusqu'au baccalaureat, et a l'age de 20 ans, il a reussi un concours pour recrutement d'administrateur dans un etablissement public. Depuis son jeune age, Monsieur A avait de l'affection pour une amie de classe, une voisine Mademoiselle Z, issue d'une famille nettement plus aisee que la sienne. A l'age de vingt-deux ans, Monsieur A fidele a ses sentiments, et a sa bien-aimee, a demande Mademoiselle Z eu mariage, demande qui n'a pas ete bien accueillie par la famille de mademoiselle Z et surtout par sa

## Version 0.2

In [20]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer

def classify_status(text):
    
    ########################################## PREPROCESSING THE TEXT #####################################

    text = preprocess_text(text)

    ########################################## LOADING A TEXT DOCUMENT ####################################

    doc = TextDocument(text=text)

    ########################################## SPLITTING A DOCUMENT IN SENTENCES ##########################

    sent_tokenizer = SentenceTokenizer(
        output_label="sentence",
        punct_chars=[".", "?", "!"],
    )

    sentences = sent_tokenizer.run([doc.raw_segment])


    ########################################## FINDING ENTITIES ###########################################

    ## On cherche les entités du tabagisme

    ####### regex_tabac
    ## On récupère  ces entités dans le cas ou:
    ##  => ces entités ne sont pas précédés de "statut tabagisme :  " 
    ## Dans ce cas, le statut tabagique du patient est inconnu



    # regex_tabac = r"(?<!statut\stabagisme\s:\s\b)(\b\s*(taba(c|gisme)|tabagique|cigare(tte)?[s]?|tabac|fumeur[s]?|fumeuse[s]?|fume(e)?[s]?|fume[r]?))\b"
    regex_tabac = r"(?<!statut\stabagisme\s:\s\b)(\b\s*(taba(c|gisme)|tabagique|cigare(tte)?[s]?|tabac|fumeur[s]?|fumeuse[s]?|fume(e)?[s]?|fume[r]?))\b"
    regex_marital = r"\b(marie[e]?|celibataire|divorce[e]?|veuf|veuve|pacse[e][s]?|concubinage|(vit|habite)\sseul(e)?)\b"
    regex_alcool = r"\b(alcool|boit|alcoolique|dependance alcool|alcoolisme)\b"
    regex_sans_antecedent = r"\bantecedent(s)?\b"

    regexp_rules = [
        RegexpMatcherRule(regexp=regex_tabac, label="statut_tabagisme"),
        RegexpMatcherRule(regexp=regex_marital, label="statut_marital"),
        RegexpMatcherRule(regexp=regex_alcool, label="statut_alcool"),
        RegexpMatcherRule(regexp=regex_sans_antecedent, label="antecedent"),

    ]

    regexp_matcher = RegexpMatcher(rules=regexp_rules)

    entities = regexp_matcher.run(sentences)


    ########################################## DETECTING NEGATION #########################################


    ## statut negatif: Si les expressions récupérées par nos regex contiennent:
    # - "non" ex: statut tabagisme: non
    # - "ancien" ex: ancien fumeur
    # - "0 boite(s)/paquet(s) ..." ex: statut tabagisme: 0 boite
    neg_rules = [
        
        ## STATUT MARITAL
        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\s*marie(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*divorce(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*celibataire(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*veu(f|ve)\b"),
        
        ## STATUT TABAGISME
        NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend)\s*pas"),
        NegationDetectorRule(regexp=r"jamais"),
        NegationDetectorRule(regexp=r"arret"),
        NegationDetectorRule(regexp=r"ancien"),
        NegationDetectorRule(regexp=r"\b:\s*non\b"),
        NegationDetectorRule(regexp=r"ne\s*fume\s*pas"),
        NegationDetectorRule(regexp=r"\b0\s*(paquet[s]?|cigarette[s]?|jr[s]?|jour[s]?|boite[s]?)\b"),
        NegationDetectorRule(regexp=r"\b(non(-|\s)?fumeur|jamais(/s)*fum)"),
        NegationDetectorRule(regexp=r"\b(non(-|\s)?tabagique)"),
        NegationDetectorRule(regexp=r"\bni\s*tabac\b"),
        
        ## STATUT ALCOOL
        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bpas\s*alcool(ique)?\b"),
        NegationDetectorRule(regexp=r"\bpas\s*d'alcool\b"),
        
        ## STATUT ALCOOL
        NegationDetectorRule(regexp=r"sans\s*antecedent(s)?"),
        NegationDetectorRule(regexp=r"\bpas\s*d'antecedent(s)?\b"),
        NegationDetectorRule(regexp=r"\baucun\s*antecedent(s)?\b"),

    ]
    
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)

    ########################################## DETECTING TRUE OR WRONG NEGATION ###########################
    
    """
    Pour etre plus précis, on va appliquer neg_detector sur des plus petites parties de la phrase
    """
    
    # On sépare les phrases si il y'a "mais" "et"
    synt_tokenizer = SyntagmaTokenizer(
        output_label="sentence",
        separators=[r"\bmais\b", r"\bet\b"],
    )
    
    syntagmas = synt_tokenizer.run(sentences)
    
    #On recherche des negations dans les plus petites phrases pour etre plus précis
    neg_detector.run(syntagmas)

    ########################################## DETECTING OTHER PATIENT OR NOT ############################

    from medkit.text.context import FamilyDetector

    # Instanciation de la classe FamilyDetector avec une étiquette de sortie spécifique
    family_detector = FamilyDetector(output_label='other_detected')

    # On applique family_detector aux syntagmas
    family_detector.run(syntagmas)
    
    ##########################################  CREATION OF ENTITIES  ####################################

    regexp_matcher = RegexpMatcher(rules=regexp_rules, attrs_to_copy=["is_negated","other_detected"])
    entities = regexp_matcher.run(syntagmas)
    
    ########################################## AUGMENTING A DOCUMENT #####################################

    for entity in entities:
        doc.anns.add((entity))

    dico = doc.to_dict()
    
    ########################################## CLASSIFICATION  ###########################################
    
    ## Par défaut
    statut_tabagisme = "UNKNOWN"
    statut_marital = "UNKNOWN"
    statut_alcool = "UNKNOWN"
    n_oui_tabac = 0
    n_non_tabac = 0
    n_oui_alcool = 0
    n_non_alcool = 0
    
    # On va classer le statut tabagique du patient en fonction des entitées trouvées et si elles sont négatives

    for ann in dico['anns']:
        # On récupère les valeurs is_negated et other_detected
        value_is_negated = ann['attrs'][0]['value']
        value_other_detected = ann['attrs'][1]['value']
        
        ## Si l'entitée trouvée ne concerne pas le patient donc other_detected == True
        if value_other_detected:
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")            
            continue

    ########################################## 1) STATUT TABAC ###########################################

        if ann['label']== "statut_tabagisme":
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            # Si c'est une négation, on incrémente n_non_tabac de 1
            if value_is_negated == True:
                n_non_tabac += 1
            # Si ce n'est pas une négation, on incrémente n_oui_tabac de 1
            else:
                n_oui_tabac += 1

    ########################################## 2) STATUT MARITAL #########################################


        if ann['label']== "statut_marital":

            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            if ann['attrs'][0]['value'] == False:
                statut_marital = ann['text']
            else:
                statut_marital = "pas"+" "+ann['text']


    ########################################## 3) STATUT ALCOOL ##########################################

        # On va classer le statut d'alcoolisme du patient en fonction des entitées trouvées et si elles sont négatives

        if ann['label']== "statut_alcool":
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            if value_is_negated == True:
                n_non_alcool += 1
            else:
                n_oui_alcool += 1

    ########################################## ANTECEDENT ##########################################

   # On va vérifier si il y'a eu des antecedents ou non

        if ann['label']== "antecedent":
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            # si il y a "sans antecedent(s)", "pas d'antecedent(s)", "aucun antecedent(s)"
            if value_is_negated == True:
                statut_tabagisme = "NON-FUMEUR"
                statut_alcool = "NON-ALCOOLIQUE"
        
            
    ########################################## CALCUL PROPORTION FUMEUR ET ALCOOL #########################
           
    ## On calcule la proportion de terme FUMEUR et ALCOOL pour choisir un statut
    # Note: si il y'a eu des termes du regex tabac ou alcool, ils l'emportent sur la présence des termes
    # "sans antecedent(s)", "pas d'antecedent(s)", "aucun antecedent(s)"
    
    p_neg_tabac = 0
    p_pos_tabac = 0
    p_neg_alcool = 0
    p_pos_alcool = 0
        
    if len(dico['anns']) != 0:
        
        dico_vide = False # Booléen, le dico contient des annotations
        
        ### Porportion TABAC ### 
        p_neg_tabac = n_non_tabac/len(dico['anns'])
        p_pos_tabac = n_oui_tabac/len(dico['anns'])
        ### Porportion ALCOOL ###
        p_neg_alcool = n_non_alcool/len(dico['anns'])
        p_pos_alcool = n_oui_alcool/len(dico['anns'])
        
    else:
        dico_vide = True
           
    ## On choisit un statut en fonction des proportions calculées 
    
    if p_pos_tabac > p_neg_tabac:
        statut_tabagisme = "FUMEUR"

    if p_pos_tabac < p_neg_tabac:
        statut_tabagisme = "NON-FUMEUR"

    if p_pos_alcool > p_neg_alcool:
        statut_alcool = "ALCOOLIQUE"
        
    if p_pos_alcool < p_neg_alcool:
        statut_alcool = "NON-ALCOOLIQUE"
                 
            
    return statut_tabagisme, statut_marital, statut_alcool, dico_vide

classify_status("Le père de Jean, âgé de 70 ans, est un fumeur régulier depuis plus de 40 ans. Le patient ne fume pas")


Texte :  fumeur, Is_negated : False, other_detected : True
Texte :  fume, Is_negated : True, other_detected : False


('NON-FUMEUR', 'UNKNOWN', 'UNKNOWN', False)

V 0.4

In [ ]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer

def classify_status(text):
    
    ########################################## PREPROCESSING THE TEXT #####################################

    text = preprocess_text(text)

    ########################################## LOADING A TEXT DOCUMENT ####################################

    doc = TextDocument(text=text)

    ########################################## SPLITTING A DOCUMENT IN SENTENCES ##########################

    sent_tokenizer = SentenceTokenizer(
        output_label="sentence",
        punct_chars=[".", "?", "!"],
    )

    sentences = sent_tokenizer.run([doc.raw_segment])


    ########################################## FINDING ENTITIES ###########################################

    ## On cherche les entités du tabagisme

    ####### regex_tabac
    ## On récupère  ces entités dans le cas ou:
    ##  => ces entités ne sont pas précédés de "statut tabagisme :  " 
    ## Dans ce cas, le statut tabagique du patient est inconnu



    # regex_tabac = r"(?<!statut\stabagisme\s:\s\b)(\b\s*(taba(c|gisme)|tabagique|cigare(tte)?[s]?|tabac|fumeur[s]?|fumeuse[s]?|fume(e)?[s]?|fume[r]?))\b"
    regex_tabac = r"(?<!statut\stabagisme\s:\s\b)(\b\s*(taba(c|gisme)|tabagique|cigare(tte)?[s]?|tabac|fumeur[s]?|fumeuse[s]?|fume(e)?[s]?|fume[r]?))\b"
    regex_marital = r"\b(marie[e]?|celibataire|divorce[e]?|veuf|veuve|pacse[e][s]?|concubinage|(vit|habite)\sseul(e)?)\b"
    regex_alcool = r"\b(alcool|boit|alcoolique|dependance alcool|alcoolisme)\b"
    regex_sans_antecedent = r"\bantecedent(s)?\b"

    regexp_rules = [
        RegexpMatcherRule(regexp=regex_tabac, label="statut_tabagisme"),
        RegexpMatcherRule(regexp=regex_marital, label="statut_marital"),
        RegexpMatcherRule(regexp=regex_alcool, label="statut_alcool"),
        RegexpMatcherRule(regexp=regex_sans_antecedent, label="antecedent"),

    ]

    regexp_matcher = RegexpMatcher(rules=regexp_rules)

    entities = regexp_matcher.run(sentences)


    ########################################## DETECTING NEGATION #########################################


    ## statut negatif: Si les expressions récupérées par nos regex contiennent:
    # - "non" ex: statut tabagisme: non
    # - "ancien" ex: ancien fumeur
    # - "0 boite(s)/paquet(s) ..." ex: statut tabagisme: 0 boite
    neg_rules = [
        
        ## STATUT MARITAL
        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\s*marie(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*divorce(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*celibataire(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*veu(f|ve)\b"),
        
        ## STATUT TABAGISME
        NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend)\s*pas"),
        NegationDetectorRule(regexp=r"jamais"),
        NegationDetectorRule(regexp=r"arret"),
        NegationDetectorRule(regexp=r"ancien"),
        NegationDetectorRule(regexp=r"\b:\s*non\b"),
        NegationDetectorRule(regexp=r"ne\s*fume\s*pas"),
        NegationDetectorRule(regexp=r"\b0\s*(paquet[s]?|cigarette[s]?|jr[s]?|jour[s]?|boite[s]?)\b"),
        NegationDetectorRule(regexp=r"\b(non(-|\s)?fumeur|jamais(/s)*fum)"),
        NegationDetectorRule(regexp=r"\b(non(-|\s)?tabagique)"),
        NegationDetectorRule(regexp=r"\bni\s*tabac\b"),
        
        ## STATUT ALCOOL
        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bpas\s*alcool(ique)?\b"),
        NegationDetectorRule(regexp=r"\bpas\s*d'alcool\b"),
        
        ## STATUT ALCOOL
        NegationDetectorRule(regexp=r"sans\s*antecedent(s)?"),
        NegationDetectorRule(regexp=r"\bpas\s*d'antecedent(s)?\b"),
        NegationDetectorRule(regexp=r"\baucun\s*antecedent(s)?\b"),

    ]
    
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)

    ########################################## DETECTING TRUE OR WRONG NEGATION ###########################
    
    """
    Pour etre plus précis, on va appliquer neg_detector sur des plus petites parties de la phrase
    """
    
    # On sépare les phrases si il y'a "mais" "et"
    synt_tokenizer = SyntagmaTokenizer(
        output_label="sentence",
        separators=[r"\bmais\b", r"\bet\b"],
    )
    
    syntagmas = synt_tokenizer.run(sentences)
    
    #On recherche des negations dans les plus petites phrases pour etre plus précis
    neg_detector.run(syntagmas)

    ########################################## DETECTING OTHER PATIENT OR NOT ############################

    from medkit.text.context import FamilyDetector

    # Instanciation de la classe FamilyDetector avec une étiquette de sortie spécifique
    family_detector = FamilyDetector(output_label='other_detected')

    # On applique family_detector aux syntagmas
    family_detector.run(syntagmas)
    
    ##########################################  CREATION OF ENTITIES  ####################################

    regexp_matcher = RegexpMatcher(rules=regexp_rules, attrs_to_copy=["is_negated","other_detected"])
    entities = regexp_matcher.run(syntagmas)
    
    ########################################## AUGMENTING A DOCUMENT #####################################

    for entity in entities:
        doc.anns.add((entity))

    dico = doc.to_dict()
    
    ########################################## CLASSIFICATION  ###########################################
    
    ## Par défaut
    statut_tabagisme = "UNKNOWN"
    statut_marital = "UNKNOWN"
    statut_alcool = "UNKNOWN"
    n_oui_tabac = 0
    n_non_tabac = 0
    n_oui_alcool = 0
    n_non_alcool = 0
    
    # On va classer le statut tabagique du patient en fonction des entitées trouvées et si elles sont négatives

    for ann in dico['anns']:
        # On récupère les valeurs is_negated et other_detected
        value_is_negated = ann['attrs'][0]['value']
        value_other_detected = ann['attrs'][1]['value']
        
        ## Si l'entitée trouvée ne concerne pas le patient donc other_detected == True
        if value_other_detected:
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")            
            continue

    ########################################## 1) STATUT TABAC ###########################################

        if ann['label']== "statut_tabagisme":
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            # Si c'est une négation, on incrémente n_non_tabac de 1
            if value_is_negated == True:
                n_non_tabac += 1
            # Si ce n'est pas une négation, on incrémente n_oui_tabac de 1
            else:
                n_oui_tabac += 1

    ########################################## 2) STATUT MARITAL #########################################


        if ann['label']== "statut_marital":

            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            if ann['attrs'][0]['value'] == False:
                statut_marital = ann['text']
            else:
                statut_marital = "pas"+" "+ann['text']


    ########################################## 3) STATUT ALCOOL ##########################################

        # On va classer le statut d'alcoolisme du patient en fonction des entitées trouvées et si elles sont négatives

        if ann['label']== "statut_alcool":
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            if value_is_negated == True:
                n_non_alcool += 1
            else:
                n_oui_alcool += 1

    ########################################## ANTECEDENT ##########################################

   # On va vérifier si il y'a eu des antecedents ou non

        if ann['label']== "antecedent":
            print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            # si il y a "sans antecedent(s)", "pas d'antecedent(s)", "aucun antecedent(s)"
            if value_is_negated == True:
                statut_tabagisme = "NON-FUMEUR"
                statut_alcool = "NON-ALCOOLIQUE"
        
            
    ########################################## CALCUL PROPORTION FUMEUR ET ALCOOL #########################
           
    ## On calcule la proportion de terme FUMEUR et ALCOOL pour choisir un statut
    # Note: si il y'a eu des termes du regex tabac ou alcool, ils l'emportent sur la présence des termes
    # "sans antecedent(s)", "pas d'antecedent(s)", "aucun antecedent(s)"
    
    p_neg_tabac = 0
    p_pos_tabac = 0
    p_neg_alcool = 0
    p_pos_alcool = 0
        
    if len(dico['anns']) != 0:
        
        dico_vide = False # Booléen, le dico contient des annotations
        
        ### Porportion TABAC ### 
        p_neg_tabac = n_non_tabac/len(dico['anns'])
        p_pos_tabac = n_oui_tabac/len(dico['anns'])
        ### Porportion ALCOOL ###
        p_neg_alcool = n_non_alcool/len(dico['anns'])
        p_pos_alcool = n_oui_alcool/len(dico['anns'])
        
    else:
        dico_vide = True
           
    ## On choisit un statut en fonction des proportions calculées 
    
    if p_pos_tabac > p_neg_tabac:
        statut_tabagisme = "FUMEUR"

    if p_pos_tabac < p_neg_tabac:
        statut_tabagisme = "NON-FUMEUR"

    if p_pos_alcool > p_neg_alcool:
        statut_alcool = "ALCOOLIQUE"
        
    if p_pos_alcool < p_neg_alcool:
        statut_alcool = "NON-ALCOOLIQUE"
                 
            
    return statut_tabagisme, statut_marital, statut_alcool, dico_vide

classify_status("Le père de Jean, âgé de 70 ans, est un fumeur régulier depuis plus de 40 ans. Le patient ne fume pas")
